In [ ]:
%pip install keras-tuner --upgrade

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import keras_tuner as kt
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
dataset = pd.read_csv("products.csv")
dataset = dataset.sample(frac=1)
dataset_features = dataset.copy()
dataset_label = dataset_features.pop('MTTF')

In [ ]:
features_max = dataset_features.max()
labels_max = dataset_label.max()
dataset_features /= features_max
dataset_label /= dataset_label.max()

In [ ]:
x, x_test, y, y_test = train_test_split(dataset_features, dataset_label, test_size=0.33)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

In [ ]:
x_train